In [2]:
!git clone https://github.com/muqtasid87/benchmark_vlm

fatal: destination path 'benchmark_vlm' already exists and is not an empty directory.


In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from transformers import BitsAndBytesConfig
from transformers.image_utils import load_image
import torch
import csv
import platform
from transformers import pipeline
import psutil
import numpy as np
from PIL import Image
import os
import shutil
import time
from codecarbon import EmissionsTracker

In [4]:
# model_id = ["Qwen/Qwen2-VL-2B-Instruct-GPTQ-Int4","Qwen/Qwen2-VL-7B-Instruct-AWQ"]
model_id = ["Qwen/Qwen2-VL-7B-Instruct-AWQ"]

In [5]:
# Image directory
img_dir = 'benchmark_vlm/master'

### Inference Function

In [6]:
# Function to run inference
def infer_image(image_path, prompt):
    with open(image_path, "rb") as f:
          input_image = Image.open(f).convert("RGB")

    
    conversation = [
          {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt},
                
              ],
          },
      ]

    text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(text=[text_prompt], images=[input_image], padding=True, return_tensors="pt").to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=2)
    generated_ids = [output_ids[len(input_ids) :]for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return output_text[0].lower()
    
    
    

### Validation function

In [7]:
# Define vehicle classes for validation
vehicle_classes = {
    "Class 1 - Motorcycles": ["motorcycle", "motorbike", "bike", "scooter", "motor", "moped"],
    "Class 2 - Car, SUV, Station Wagons": ["car", "sedan", "hatchback", "suv", "mpv", "crossover", "station wagon", "wagon", 'coupe', 'convertible', 'sports car'],
    "Class 3 - 2 axle lorry, van, pickup": ["pickup", "van", "minivan"],
    "Class 4 - 2 axles, 5 or 6 wheels bus or truck": ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 5 - 3 axle truck, bus": ["bus", "truck"],
    "Class 6 - 4 axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 7 - 5 or more axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "bus 2" : ["coach", "bus"],
    "bus 3" : ["coach", "bus"],
    "truck 2" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 3" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 4" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 5" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "van" : ["van", "minivan", 'minibus',"mpv"],
    "pickup" : ["pickup", "pickup truck", "camper"],
}

valid_vehicle_types = {type for sublist in vehicle_classes.values() for type in sublist}

In [8]:
# Validate predictions

def validate_prediction(folder_name, prediction, axles=None):
    # Extract folder label
    for class_name, valid_labels in vehicle_classes.items():
        if folder_name.startswith(class_name):
            if axles:  # Use axle count for bus/truck
                if axles == 1 or 0:
                  return False
                elif axles == 2:
                    return "2" in folder_name[-1]
                elif axles == 3:
                    return "3" in folder_name[-1]
                elif axles == 4:
                    return "4" in folder_name[-1]
                elif axles >= 5:
                    return "5" in folder_name[-1]
            if isinstance(valid_labels, list):
                return prediction in valid_labels
            return prediction == valid_labels
    return False

# Main Loop

In [9]:
from tqdm import tqdm

In [10]:
def main_loop(wrong_predictions_dir, img_dir=img_dir, use_axles=True, valid_vehicle_types=valid_vehicle_types):
    total_files = sum(len(files) for _, _, files in os.walk(img_dir))
    progress_bar = tqdm(total=total_files, desc="Processing Images", unit="file")
    # Main processing loop
    start_time = time.time()
    tracker = EmissionsTracker()
    tracker.start()
    results = []
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith((".jpg", ".png", ".jpeg")):
                image_path = os.path.join(root, file)
                folder_name = os.path.basename(os.path.dirname(image_path))
                progress_bar.set_description(f"Processing: {folder_name}")
                progress_bar.update(1)  # Increment the progress bar

                try:
                    # First inference: vehicle type
                    # vehicle_prompt = f"<image> answer en What type of vehicle is this? Answer in only one word."
                    vehicle_prompt = f"What type of vehicle is this? Choose one from: {', '.join(valid_vehicle_types)}. Answer in one word"
                    prediction = infer_image(image_path, vehicle_prompt)
                    print(f"Vehicle Prediction: {prediction}")

                    # Handle buses and trucks
                    axles = None
                    if prediction in ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck',  "tow truck", "dump trailer"]:
                        # Check if it's a pickup truck
                        if prediction == 'truck':
                            pickup_prompt = "Is this a pickup truck? Respond with only yes or no."
                            is_pickup = infer_image(image_path, pickup_prompt).strip().lower()
                            print(f"Is Pickup Truck: {is_pickup}")
                            if is_pickup == "yes":
                                prediction = "pickup truck"
                            elif use_axles:
                                # Predict axles for non-pickup trucks
                                axle_prompt = "How many axles does this vehicle have? Answer with a single integer value."
                                axle_prediction = infer_image(image_path, axle_prompt)
                                print(f"Axle Prediction: {axle_prediction}")
                    
                                try:
                                    axles = int(axle_prediction)
                                except ValueError:
                                    print(f"Invalid axle prediction: {axle_prediction}")
                                    axles = None  # Fallback if parsing fails
                        elif use_axles:

                            axle_prompt = "How many axles does this vehicle have? Answer with a single integer value"
                            axle_prediction = infer_image(image_path, axle_prompt)
                            print(f"Axle Prediction: {axle_prediction}")
    
                            try:
                                axles = int(axle_prediction)
                            except ValueError:
                                print(f"Invalid axle prediction: {axle_prediction}")

                    # Validate prediction
                    is_correct = validate_prediction(folder_name, prediction, axles)
                    print(f"Validation Result: {is_correct}")

                    # Store results
                    results.append({
                        "image": image_path,
                        "folder_label": folder_name,
                        "prediction": prediction,
                        "axles": axles,
                        "is_correct": is_correct
                    })

                    # Handle wrong predictions
                    if not is_correct:
                        # Create a folder for the wrong prediction
                        wrong_folder = os.path.join(wrong_predictions_dir, prediction, str(axles))
                        os.makedirs(wrong_folder, exist_ok=True)

                        # Copy the image to the folder
                        shutil.copy(image_path, os.path.join(wrong_folder, file))
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
                    continue
    
    progress_bar.close()
    emissions = tracker.stop()
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processing time: {elapsed_time:.2f} seconds")
    print(f"CO2 emissions: {emissions:.6f} kg")
    return (results, elapsed_time, emissions)

In [11]:
# Save results to CSV
def export_results(results, model_id, elapsed_time, emissions):
# Save results to CSV
    output_csv = f'experiment_data/{models}/axles/results.csv'
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["image", "folder_label", "prediction", "axles", "is_correct"])
        writer.writeheader()
        writer.writerows(results)

    # Metrics calculation
    correct_predictions = sum(1 for r in results if r["is_correct"])
    total_images = len(results)
    accuracy = correct_predictions / total_images * 100 if total_images > 0 else 0
    wrong_predictions = total_images - correct_predictions

    # Per-class metrics
    class_metrics = {}
    for result in results:
        label = result["folder_label"]
        correct = result["is_correct"]
        if label not in class_metrics:
            class_metrics[label] = {"correct": 0, "total": 0}
        class_metrics[label]["total"] += 1
        if correct:
            class_metrics[label]["correct"] += 1

    # Aggregate class metrics
    per_class_accuracy = {
        label: (data["correct"] / data["total"]) * 100
        for label, data in class_metrics.items()
    }
    per_class_error_rate = {
        label: 100 - acc for label, acc in per_class_accuracy.items()
    }

    # Collect system configuration
    system_config = {
        "Processor": platform.processor(),
        "CPU Count": psutil.cpu_count(logical=True),
        "Memory (GB)": round(psutil.virtual_memory().total / (1024 ** 3), 2),
        "Elapsed Time (seconds)": elapsed_time,
        "CO2 Emissions (kg)": round(emissions, 6)
    }

    # Metrics dictionarya
    metrics = {
        "Total Images": total_images,
        "Correct Predictions": correct_predictions,
        "Wrong Predictions": wrong_predictions,
        "Accuracy (%)": accuracy,
        "Error Rate (%)": 100 - accuracy
    }

    # Save metrics to CSV
    metrics_csv = f'experiment_data/{models}/axles/metrics.csv'
    with open(metrics_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Metric", "Value"])
        for key, value in metrics.items():
            writer.writerow([model_id, key, value])
        for label, acc in per_class_accuracy.items():
            writer.writerow([model_id, f"Per-Class Accuracy: {label}", acc])
            writer.writerow([model_id, f"Per-Class Error Rate: {label}", per_class_error_rate[label]])
        for key, value in system_config.items():
            writer.writerow([model_id, key, value])

    # Print final metrics
    print("\n--- Metrics ---")
    for key, value in metrics.items():
        print(f"{key}: {value}")
    for label, acc in per_class_accuracy.items():
        print(f"Class '{label}' - Accuracy: {acc:.2f}%, Error Rate: {per_class_error_rate[label]:.2f}%")
    for key, value in system_config.items():
        print(f"{key}: {value}")

In [12]:
for models in model_id:
    print(models)
    model = Qwen2VLForConditionalGeneration.from_pretrained(models, torch_dtype=torch.bfloat16, device_map="auto").eval()
    min_pixels = 256*28*28
    max_pixels = 1280*28*28
    processor = AutoProcessor.from_pretrained(models, min_pixels=min_pixels, max_pixels=max_pixels) #

    wrong_predictions_dir = f'experiment_data/{models}/axles/wrong_predictions'  # Directory to store wrongly predicted images
    os.makedirs(wrong_predictions_dir, exist_ok=True)
    run = main_loop(wrong_predictions_dir = wrong_predictions_dir)
    export_results(run[0], models, run[1], run[2])
    
        

Qwen/Qwen2-VL-7B-Instruct-AWQ


We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/932 [00:00<?, ?file/s][codecarbon INFO @ 11:04:12] [setup] RAM Tracking...
[codecarbon INFO @ 11:04:12] [setup] CPU Tracking...
[codecarbon WARNING @ 11:04:12] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 11:04:13] CPU Model on constant consumption mode: AMD EPYC 7513 32-Core Processor
[codecarbon INFO @ 11:04:13] [setup] GPU Tracking...
[codecarbon INFO @ 11:04:13] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:04:13] >>> Tracker's metadata:
[codecarbon INFO @ 11:04:13]   Platform system: Linux-5.15.0-125-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 11:04:13]   Python version: 3.11.10
[codecarbon INFO @ 11:04:13]   CodeCarbon version: 2.8.2
[codecarbon INFO @ 11:04:13]   Available RAM : 503.471 GB
[codecarbon INFO @ 11:04:13]   CPU count: 128
[codecarbon INFO @ 11:04:13]   CPU model: AMD EPYC 7513 32-

Vehicle Prediction: motorbike
Validation Result: True


...

Axle Prediction: 6
Validation Result: True
Vehicle Prediction: dump truck


Processing: truck 5:  99%|█████████▉| 927/932 [14:57<00:06,  1.34s/file]

Axle Prediction: 6
Validation Result: True
Vehicle Prediction: truck
Is Pickup Truck: no


Processing: truck 5:  99%|█████████▉| 927/932 [14:58<00:04,  1.03file/s]
[codecarbon INFO @ 11:19:11] Energy consumed for RAM : 0.046883 kWh. RAM Power : 188.80178117752075 W
[codecarbon INFO @ 11:19:11] Energy consumed for all CPUs : 0.024834 kWh. Total CPU Power : 100.0 W
[codecarbon INFO @ 11:19:11] Energy consumed for all GPUs : 0.076664 kWh. Total GPU Power : 305.5347751449195 W
[codecarbon INFO @ 11:19:11] 0.148381 kWh of electricity used since the beginning.


Axle Prediction: 6
Validation Result: True
Total processing time: 898.80 seconds
CO2 emissions: 0.066730 kg

--- Metrics ---
Total Images: 927
Correct Predictions: 646
Wrong Predictions: 281
Accuracy (%): 69.68716289104638
Error Rate (%): 30.312837108953616
Class 'Class 1 - Motorcycles' - Accuracy: 92.00%, Error Rate: 8.00%
Class 'Class 2 - Car, SUV, Station Wagons' - Accuracy: 97.98%, Error Rate: 2.02%
Class 'pickup' - Accuracy: 82.00%, Error Rate: 18.00%
Class 'van' - Accuracy: 100.00%, Error Rate: 0.00%
Class 'bus 2' - Accuracy: 69.05%, Error Rate: 30.95%
Class 'truck 2' - Accuracy: 78.22%, Error Rate: 21.78%
Class 'bus 3' - Accuracy: 27.55%, Error Rate: 72.45%
Class 'truck 3' - Accuracy: 33.00%, Error Rate: 67.00%
Class 'truck 4' - Accuracy: 32.18%, Error Rate: 67.82%
Class 'truck 5' - Accuracy: 79.00%, Error Rate: 21.00%
Processor: x86_64
CPU Count: 128
Memory (GB): 503.47
Elapsed Time (seconds): 898.7975969314575
CO2 Emissions (kg): 0.06673
